# OpenAI - Caribbean Challenge Competition 

### Train an Image Classifier from Aerial Photos with Transfer learning 

### CNN2d - 4layers



 **modify the preprocessing image.py file**

- see: https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085

- www.github.com/rockyxu66/Kaggles_Flowers_Classification_Keras


Add skimage preprocessing techniques:

- Histogram Equalization

- Contrast Stretching

- Adaptive Equalization

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

import cv2
from skimage import data, img_as_float
from skimage import exposure

In [ ]:
# TF Keras imports
import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D, GlobalAveragePooling2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import *
from keras.activations import *
from keras.layers import *
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.utils import Sequence
from keras.utils import to_categorical


In [ ]:
from keras import backend as K
# from tensorflow.keras import backend as K

config = tf.ConfigProto()
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
sess = tf.Session(config=config)

In [ ]:
def preprocess_input(x):
    x /= 256
    x -= 0.5
    return x


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

def dice_coef_loss(y_true, y_pred):
#     return -dice_coef(y_true, y_pred)
    return 1-dice_coef(y_true, y_pred)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [ ]:
def get_image_id_from_path(image_path):
    """ returns image id from image path """
    return image_path.split('./data/processed2/trainImages/')[0].split('.png')[0].split('-')[0][-8:]


def get_label_from_id(image_id):
    """ returns label from image id """
    return np.array(labels_org[labels_org.id==image_id].iloc[:,2:]).argmax()

### Load image data

In [ ]:
curr_path = Path('/home/ime/Documents/PycharmProjects/DrivenData/OpenAI')
data_path = curr_path / 'data/raw'
train_path = curr_path / 'data/pickle/'
img_path = curr_path / 'data/processed2/trainImages/'

categories = ['concrete_cement', 'healthy_metal', 'incomplete', 'irregular_metal', 'other']

In [ ]:
# Load all image file names
fnames = []
for categ in categories:
    sub_folder = os.path.join(img_path, categ)
    file_names = os.listdir(sub_folder)
    full_path = [os.path.join(sub_folder, file_name) for file_name in file_names]
    fnames.append(full_path)

In [ ]:
print('length of each class:', [len(f) for f in fnames])
print('total number of images:', sum([len(f) for f in fnames]))

In [ ]:
# Load train_labels

labels_org = pd.read_csv(data_path/'train_labels.csv')

labels_org.head()

In [ ]:
one_img_id = get_image_id_from_path(fnames[3][30])
get_label_from_id(one_img_id)

### Load images

In [ ]:
images = []

for names in fnames:
    one_class_img = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_class_img)

del one_class_img
gc.collect()

In [ ]:
# # # Load images with keras

# images2 = []
# for names in fnames:
#     one_class_img = [image.load_img(name) for name in names if (image.load_img(name)) is not None]
#     images2.append(one_class_img)


In [ ]:
print('no. of images for each class:', [len(f) for f in images])
# print('no. of images for each class:', [len(f) for f in images2 if images2 is not None])

### Calculate minimal shape for all images

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(6, 18))
for i, imgs in enumerate(images):
    shapes = [img.shape for img in imgs]
    widths = [shape[0] for shape in shapes]
    heights = [shape[1] for shape in shapes]
    print('%d, %d is the min shape for %s' % (np.min(widths), np.min(heights), categories[i]))    
    print()
    print('%d, %d is the max shape for %s' % (np.max(widths), np.max(heights), categories[i]))    
    print('-'*10)
    ax[i].scatter(widths, heights, label=('class %s' % i))
    ax[i].set_title(('class %s' % i))
#     print('-'*10)
#     ax[i].scatter(widths, heights, label=('class %s' % i))
#     ax[i].set_title(('class %s' % i))

### Show sample of images for all classes

In [ ]:
plt.figure(figsize=(15,10))
for i, imgs in enumerate(images):
    plt.subplot(2, 3, i+1)
    idx = np.random.randint(len(imgs))
    plt.imshow(imgs[idx])
    plt.grid('off')
    plt.title(categories[i]+' '+str(idx))
plt.show()

### Resize all images

In [ ]:
TRAIN_ONESIZE = True     # train with a unified image size
# TRAIN_ONESIZE = False    # train with original image size


IMG_SIZE = 224
NO_CHANNELS = 3

if TRAIN_ONESIZE:
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
else:
     input_shape = (None, None, 3)

In [ ]:
# apply to all images

images_resized = []
for i, imgs in enumerate(images):
    images_resized.append([cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA) for img in imgs])

In [ ]:
rand_categ = 2   # select 0-4

rand_idx = np.random.randint(0, len(images[rand_categ]))

print('class:', categories[rand_categ])
print()
print('before:', images[rand_categ][rand_idx].shape)
print('after:', images_resized[rand_categ][rand_idx].shape)

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.title('original image')
plt.grid('False')
plt.imshow(images[rand_categ][rand_idx])
plt.subplot(1,2,2)
plt.title('resized image')
plt.grid('False')
plt.imshow(images_resized[rand_categ][rand_idx])
plt.show()

### Check Image Preprocesing Techniques

In [ ]:
# pick a random resized image
rand_idx = np.random.randint(0,1387)
tmp_img = images_resized[0][rand_idx]

# Contrast stretching
plow, phigh = np.percentile(tmp_img, (5, 95))
tmp_img_rescale = exposure.rescale_intensity(tmp_img, in_range=(plow, phigh))

# Equalization
tmp_img_eq = exposure.equalize_hist(tmp_img)

# Adaptive Equalization
tmp_img_adapteq = exposure.equalize_adapthist(tmp_img, clip_limit=0.03)

In [ ]:
# Display results
fig = plt.figure(figsize=(16, 10)) 
axes = np.zeros((2, 4), dtype=np.object)
axes[0, 0] = fig.add_subplot(2, 4, 1)

# tuple to select colors of each channel line
colors = ("r", "g", "b")
channel_ids = (0, 1, 2)

for i in range(1, 4):
    axes[0, i] = fig.add_subplot(2, 4, 1+i, sharex=axes[0,0], sharey=axes[0,0])
    
for i in range(0, 4):
    axes[1, i] = fig.add_subplot(2, 4, 5+i)

# 1-Original low res image
plt.subplot(2,4,1)
plt.title('original image')
plt.grid('False')
plt.imshow(tmp_img)

plt.subplot(2,4,5)
plt.legend(['r', 'g', 'b'])
plt.grid('False')
for channel_id, c in zip(channel_ids, colors):
        hist, bin_edges = np.histogram(tmp_img[:, :, channel_id], bins=256, range=(0, 256))
        plt.plot(bin_edges[0:-1], hist, color=c)
        plt.xlabel("Color value")
        plt.ylabel("Pixels")
        plt.xlim([0, 256])

# 2-Contrast stretching
plt.subplot(2,4,2)
plt.title('Contrast stretching')
plt.grid('False')
plt.imshow(tmp_img_rescale)

plt.subplot(2,4,6)
plt.legend(['r', 'g', 'b'])
plt.grid('False')
for channel_id, c in zip(channel_ids, colors):
        hist, bin_edges = np.histogram(tmp_img_rescale[:, :, channel_id], bins=256, range=(0, 256))
        plt.plot(bin_edges[0:-1], hist, color=c)
        plt.xlabel("Color value")
        plt.ylabel("Pixels")
        plt.xlim([0, 256])


# 3-Histogram equalization
plt.subplot(2,4,3)
plt.title('Histogram equalization')
plt.grid('False')
plt.imshow(tmp_img_eq)

plt.subplot(2,4,7)
plt.legend(['r', 'g', 'b'])
plt.grid('False')
for channel_id, c in zip(channel_ids, colors):
        hist, bin_edges = np.histogram(tmp_img_eq[:, :, channel_id], bins=256, range=(0, 256))
        plt.plot(bin_edges[0:-1], hist, color=c)
        plt.xlabel("Color value")
        plt.ylabel("Pixels")
        plt.xlim([0, 256])


# 4-Adaptive equalization
plt.subplot(2,4,4)
plt.title('Adaptive equalization')
plt.grid('False')
plt.imshow(tmp_img_adapteq)

plt.subplot(2,4,8)
plt.legend(['r', 'g', 'b'])
plt.grid('False')
for channel_id, c in zip(channel_ids, colors):
        hist, bin_edges = np.histogram(tmp_img_adapteq[:, :, channel_id], bins=256, range=(0, 256))
        plt.plot(bin_edges[0:-1], hist, color=c)
        plt.xlabel("Color value")
        plt.ylabel("Pixels")
        plt.xlim([0, 256])


fig.tight_layout()
plt.show()

In [ ]:
# del images, tmp_img_rescale, tmp_img, tmp_img_eq, tmp_img_adapteq
gc.collect()

In [ ]:
del images_eq
gc.collect()

### Apply hist equalization + stretching to all resized images

In [ ]:
images_eq = []

for cl, imgs in enumerate(images_resized):
    images_eq.append([exposure.rescale_intensity(img, in_range=(np.percentile(img, (5, 95))[0], np.percentile(img, (5, 95))[1])) for img in imgs])
    
len(images_eq)  

In [ ]:
print([len(f) for f in images_eq])

In [ ]:
# del images_resized
# gc.collect()

### Split to train/val sets

In [ ]:
train_images = []
val_images = []
train_y = []
val_y = []
# for imgs in images_resized:
for imgs in images_eq:  
    train, val = train_test_split(imgs, test_size=0.2)
    train_images.append(train)
    val_images.append(val)

### Create labels from scratch

In [ ]:
# A) kernel approach
len_train_images= [len(imgs) for imgs in train_images]
print(len_train_images)
print('sum of train images: ', np.sum(len_train_images))
train_categories = np.zeros(np.sum(len_train_images), dtype='uint8')
for i in range(5):
    if i is 0:
        train_categories[:len_train_images[i]] = i
    else:
        train_categories[np.sum(len_train_images[:i]):np.sum(len_train_images[:i+1])] = i

len_val_images = [len(imgs) for imgs in val_images]
print(len_val_images)
print('sum of val images: ', np.sum(len_val_images))
val_categories = np.zeros(np.sum(len_val_images), dtype='uint8')
for i in range(5):
    if i is 0:
        val_categories[:len_val_images[i]] = i
    else:
        val_categories[np.sum(len_val_images[:i]):np.sum(len_val_images[:i+1])] = i
 


### Convert images to arrays

In [ ]:
tmp_train_imgs = []
tmp_val_imgs = []
for imgs in train_images:
    tmp_train_imgs += imgs
    
for imgs in val_images:
    tmp_val_imgs += imgs
    
train_data = np.array(tmp_train_imgs)    
val_data = np.array(tmp_val_imgs)

In [ ]:
del tmp_train_imgs, tmp_val_imgs
gc.collect()

In [ ]:

# tmp_train = []
# tmp_val = []
# for imgs in train_images:
#     tmp_trainn = image.img_to_array(imgs)
#     train_images2.append(tmp_train)


In [ ]:
print('Before converting')
print('train data:', train_data.shape)
print('train labels:', train_categories.shape)
# print('train labels2:', train_y.shape)

train_data = train_data.astype('float32')
val_data = val_data.astype('float32')

train_labels = to_categorical(train_categories, len(categories))
val_labels = to_categorical(val_categories, len(categories))

print()
print('After converting')
print('train data:', train_data.shape)
print('train labels:', train_labels.shape)
# print('train labels2:', train_y.shape)


### Shuffle dataset (both features and labels using same seed)

In [ ]:
SEED = 100
np.random.seed(SEED)
np.random.shuffle(train_data)

np.random.seed(SEED)
np.random.shuffle(train_labels)

np.random.seed(SEED)
np.random.shuffle(val_data)

np.random.seed(SEED)
np.random.shuffle(val_labels)

In [ ]:
# for BATCH_SIZE=128 
# train_data = train_data[:11136]         # discard 14
# train_labels = train_labels[:11136]
# val_data = val_data[:3712]              # discard 8
# val_labels = val_labels[:3712]
print('shape of train data:', train_data.shape)
print('shape of train labels:', train_labels.shape)
print('shape of val data:', val_data.shape)
print('shape of val labels:', val_labels.shape)

### Synthetic data - sampling 

In [ ]:
# ### TODO: check with SMOTE
# from imblearn.over_sampling import SMOTE, ADASYN

# ada = SMOTE(random_state=42)
# train_data_res, train_labels_res = ada.fit_resample(train_data, train_labels)
# # val_data_res, val_labels_res = ada.fit_resample(val_data, val_labels)


### Master params

In [ ]:
BATCH_SIZE = 64
EPOCHS = 200

no_classes = 5

In [ ]:
# # load weights (if internet NA)
# WEIGHTS_PATH = curr_path/'saved_models/CNN/wheights_best.hdf5'

# model = load_model(path+'model.hdf5')  
# model.load_weights(WEIGHTS_PATH)   

# str(WEIGHTS_PATH).endswith('.h5')

### Build Model

In [ ]:
# WEIGHTS_PATH# Build classifier head with FC layers 

def build_cnn_model():
    inp = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    x = Conv2D(32, (3, 3), padding='same')(inp)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(96, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(96, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(32, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    
    head = GlobalAveragePooling2D()(x)
    head = Dropout(0.5)(head)
#     x = Flatten()(x)
#     x = Dense(1024)(x)
#     x = Activation('relu')(x)
#     x = Dropout(0.5)(x)
    out = Dense(no_classes, activation='softmax')(head)

    model = Model(inputs=[inp], outputs=[out])
    return model


# # Build classifier head with Avg.Pool 

#     out = Dense(5, activation='softmax')(head)

In [ ]:
model = build_cnn_model()
# model = build_resenet_head()

model.summary()

In [ ]:
# split = 142   # 105, 125, 128, 132, 142

# for layer in model.layers[:split]:
#     layer.trainable = False
    
    
# for layer in model.layers[split:]:
#     layer.trainable = True    

In [ ]:
# pd.set_option('display.max_rows', None)

# #check trainable layers
# layers_df = pd.DataFrame([(layer, layer.name, layer.trainable) for layer in model.layers], 
#                          columns=['Layer Type', 'Layer Name', 'Trainable'])

# layers_df

**Compile the final model after freezing weights**   

In [ ]:
from rectified_adam import RectifiedAdam
from AdamW import AdamW

# radam = RectifiedAdam(lr=1e-3)
# adamw = AdamW(lr=0.001)

# optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# optimizers.RMSprop(lr=2e-5)
# optimizers.Adam(lr=0.0001)    

# metrics: 'auc',log_loss    

model.compile(loss=dice_coef_loss, 
              optimizer=Adam(),                    
              metrics=['accuracy', 'categorical_crossentropy']) 

# model.compile(loss='categorical_crossentropy',       
#               optimizer=RectifiedAdam(lr=1e-3),                    
#               metrics=['accuracy']) 

In [ ]:
# callbacks 

mdl_dir = '/home/ime/Documents/PycharmProjects/DrivenData/OpenAI/saved_models/model3/'      
mdl_name = 'CNN2d_adam_dice_{epoch:02d}-{val_loss:.2f}.h5'   

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='auto') 
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4) 
mc = ModelCheckpoint(filepath=mdl_dir+mdl_name, monitor='val_loss', save_best_only=False, mode='auto')

# tb = TensorBoard(log_dir=tb_dir, write_graph=True, update_freq='epoch')

callback_list=[es, rlr, mc]

In [ ]:
# # class weights
# from sklearn.utils.class_weight import compute_class_weight

# classWeight = compute_class_weight('balanced', np.unique(train_data), train_labels)   # train_categories
# classWeight = dict(enumerate(classWeight))

# files_per_class = [len(f) for f in fnames]
# total_files = sum(files_per_class)
# classWeight2 = {}
# for i in range(len(files_per_class)):
#     classWeight2[i] = 1 - (float(files_per_class[i]) / total_files)
# #     classWeight2[i] = (float(files_per_class[i]) / total_files)
# classWeight2

In [ ]:
import math


def create_class_weight(labels_dict, mu=0.7):
    """
    labels_dict : {ind_label: count_label}
    mu : parameter to tune 
    """
    total = sum([len(f) for f in fnames])     # 14870
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight

# labels_dict
labels_dict = {0: 1387, 1: 7381, 2: 668, 3: 5241, 4: 193}
classWeight2 = create_class_weight(labels_dict)

### Create Generators

In [ ]:
# # custom IDG
# from IDG import *
# MyImageDataGenerator
# train_datagen = MyImageDataGenerator(
#     rescale=1./255,
#     contrast_stretching=True, 
#     histogram_equalization=False,
#     adaptive_equalization=False, 
# #     rotation_range=10,
# #     width_shift_range=0.1,
# #     height_shift_range=0.1,
# #     shear_range=0.2,
# #     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # validation configuration: rescale only
# val_datagen = ImageDataGenerator(rescale=1./255)

# # Flow images 
# train_generator = train_datagen.flow(train_data, train_labels, batch_size=BATCH_SIZE)
# val_generator = val_datagen.flow(val_data, val_labels, batch_size=BATCH_SIZE)

In [ ]:
# train configuration: rescale, rotation, shift, shear/zoom range, horizontal flip
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
#     shear_range=0.2,
#     zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# validation configuration: rescale only
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow images 
train_generator = train_datagen.flow(train_data, train_labels, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(val_data, val_labels, batch_size=BATCH_SIZE)

### Show generated augmented images

In [ ]:
img_id = 6203

# 1208 -- class 0
# 1205 -- class 1
# 2500, 1200 -- class 3           

img_generated = train_datagen.flow(train_data[img_id:img_id+1], train_labels[img_id:img_id+1], batch_size=1) 
img_gen = [next(img_generated) for i in range(0,5)] 

fig, ax = plt.subplots(1,5, figsize=(16, 6))
print('Labels:', [np.argmax(item[1][0]) for item in img_gen]) 
l = [ax[i].imshow(img_gen[i][0][0]) for i in range(0,5)]

## Train the model


#### Fit the model with generator

In [ ]:
print('Training: ' + mdl_name)
print()

hist = model.fit_generator(
    generator=train_generator,
    steps_per_epoch = np.ceil(len(train_data)/BATCH_SIZE),     # 349 step*BATCH_SIZE = num of images
    epochs=EPOCHS,
    validation_steps=np.ceil(len(val_data)/BATCH_SIZE),        # 117 step*BATCH_SIZE = num of images
    validation_data=val_generator,
    verbose=1,
    callbacks=callback_list,
    class_weight=classWeight2,
)

In [ ]:
# print('Training: ' + mdl_name)

# hist = model.fit_generator(
#     generator=train_generator,
#     steps_per_epoch = np.ceil(len(train_data)/BATCH_SIZE),     # 349 step*BATCH_SIZE = num of images
#     epochs=EPOCHS,
#     validation_steps=np.ceil(len(val_data)/BATCH_SIZE),        # 117 step*BATCH_SIZE = num of images
#     validation_data=val_generator,
#     verbose=1,
#     callbacks=callback_list,
#     class_weight=classWeight,
# )

In [ ]:
# Fit without generator
train_data = train_data/255.
val_data = val_data/255.

hist2 = model.fit(train_data, train_labels, 
                  batch_size=BATCH_SIZE, 
                 epochs=EPOCHS,
                 validation_data=(val_data, val_labels),
                 callbacks=callback_list,
                 class_weight=classWeight,
                 shuffle=TTrue)


### Evaluate the model

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'g')
plt.xticks()
plt.title('loss')
plt.legend(['train', 'val'], loc='best')    

plt.subplot(1,2,2)
plt.plot(hist.history['acc'], 'r')
plt.plot(hist.history['val_acc'], 'g')
plt.xticks()
plt.title('accuracy')
plt.legend(['train', 'val'], loc='best')            
plt.show()                

In [ ]:
model.evaluate(x=val_data, y=val_labels, batch_size=BATCH_SIZE) 

#### Load saved model from checkpoint

In [ ]:
print(mdl_dir, mdl_name)

model_sav = load_model(mdl_dir+mdl_name, custom_objects={'RectifiedAdam':RectifiedAdam)  #  compile=False

model_sav.summary

In [ ]:
model_sav.evaluate(x=val_data, y=val_labels, batch_size=BATCH_SIZE) 

### Predict on validation data

In [ ]:
def predict_val_data(val_data, model=model):
    """
    Returns: pred (,5)
             pred_class [0-4]
             pred_prob [0. - 1.]
    """
    val_input = np.reshape(val_data, (-1, IMG_SIZE, IMG_SIZE, 3))
    val_input = val_input/255.
    pred = model.predict(val_input)
    pred_class = np.argmax(pred, axis=1)
    return pred, pred_class, np.max(pred)  


In [ ]:
def return_categ_name(label_arr):
    idx = np.where(label_arr == 1)
    return idx[0][0]

In [ ]:
plt.figure(figsize=(10,20))
for i in range(10):
    idx = np.random.randint(len(val_data))

    ax = plt.subplot(5, 2, i+1)
    plt.imshow(val_data.astype('uint8')[idx])
    category_idx = return_categ_name(val_labels[idx])

    _, pred_class, pred_prob = predict_val_data(val_data[idx], model=model)
    
    plt.title('True: %s | Pred: %s %d%%' % (categories[category_idx], categories[pred_class], round(pred_prob, 2)*100))
    plt.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.show()

In [ ]:
y_pred_val, y_pred_class_val, _ =  predict_val_data(val_data)

In [ ]:
# np.argmax(y_pred_val, axis=1)[:10], np.argmax(val_labels, axis=1)[:10]

In [ ]:
# import scikitplot as skplt
# from scikitplot.metrics import plot_confusion_matrix
# from matplotlib.ticker import MultipleLocator

# # fig, ax  = plt.subplots(figsize=(7,7))
# plot_confusion_matrix(np.argmax(val_labels, axis=1), np.argmax(y_pred_val, axis=1), 
#                       labels=[0,1,2,3,4],  
#                       true_labels=[0,1,2,3,4], 
#                       pred_labels=[0,1,2,3,4],
# #                       figsize=(10,10),
#                       normalize=True)

# # ax.xaxis.set_major_locator(MultipleLocator(1))
# # ax.yaxis.set_major_locator(MultipleLocator(1))
# plt.imshow(cmat)
# plt.xticks(np.arange(len(categories)), categories, rotation=45)
# plt.yticks(np.arange(len(categories)), categories)
# plt.tight_layout()
# plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(np.argmax(val_labels, axis=1), np.argmax(y_pred_val, axis=1), labels=[0,1,2,3,4])

cmat   # normalize=True

In [ ]:
# # Logloss score
# score_logloss = log_loss(val_data, np.argmax(y_pred_val))
# print('Score on val data:', score_logloss)
# np.argmax(y_pred_val)
# y_pred_val[0]
# val_data[0]

In [ ]:
# # save model (optional)
# model.save('Resnet50-1024-512_loss_xxx.h5')

In [ ]:
# check missclassied images

def show_mislabeled_images(class_names, test_images, test_labels, pred_labels):
    """
        Print 25 examples of mislabeled images by the classifier, e.g when test_labels != pred_labels
    """
    BOO = (test_labels == pred_labels)
    mislabeled_indices = np.where(BOO == 0)
    mislabeled_images = test_images[mislabeled_indices]
    mislabeled_labels = pred_labels[mislabeled_indices]
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Some examples of mislabeled images by the classifier:", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(mislabeled_images[i], cmap=plt.cm.binary)
#         plt.xlabel(class_names[mislabeled_labels[i]])
    plt.show()

show_mislabeled_images(categories, val_data, val_labels, y_pred_class_val)

In [ ]:
BOO = (val_labels == y_pred_class_val)
mislabeled_indices = np.where(BOO == 0)
mislabeled_images = val_data[mislabeled_indices]
mislabeled_labels = val_labels[mislabeled_indices]

In [ ]:
val_data.shape, mislabeled_images.shape

In [ ]:
len(mislabeled_labels)

In [ ]:
# # predictions = model.predict(val_data)
# # pred_labels = np.argmax(predictions, axis = 1)

# show_mislabeled_images(categories, val_data, val_labels, y_pred_class_val)

### Predict on Test Images

### Predict one Image

In [ ]:
def predict_one_image(img, model=model):
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    img = np.reshape(img, (1, IMG_SIZE, IMG_SIZE, 3))
    img = img/255.
    pred = model.predict(img)
    pred_class = np.argmax(pred)
    return pred_class, np.max(pred)    

In [ ]:
# sample_img = cv2.imread(test_fnames[23])

# pred, prob = predict_one_image(sample_img, model)
# print('%s %d%%' % (categories[pred], round(prob,2)*100))
# _, ax = plt.subplots(1)
# plt.imshow(sample_img)
# ax.set_yticklabels([])
# ax.set_xticklabels([])
# plt.grid('off')
# plt.show()

# del pred, prob
# gc.collect()

#### Load test images filenames

In [ ]:
PREDICT_WITH_GEN = True

In [ ]:
test_fnames = []
test_file_names = os.listdir(curr_path / 'data/processed2/testImages/')
test_fnames = [os.path.join(curr_path / 'data/processed2/testImages/', file_name) for file_name in test_file_names]

In [ ]:
test_fnames[:1], test_file_names[:1]

In [ ]:
# load test Images with CV2
testImages = [cv2.imread(name) for name in test_fnames if (cv2.imread(name)) is not None]

# # load test Images with Keras
# testImages2 = [image.img_to_array(name) for name in test_fnames if (image.img_to_array(name)) is not None]

len(testImages)

In [ ]:
# check original sizes of test Images 
test_shapes = [img.shape for img in testImages]
test_widths= [shape[0] for shape in test_shapes]
test_heights = [shape[1] for shape in test_shapes]
print('%d, %d is the min shape for test set' % (np.min(test_widths), np.min(test_heights)))    
print('-'*40)
print('%d, %d is the max shape for test set' % (np.max(test_widths), np.max(test_heights))) 

#### Resize test images

In [ ]:
# check size before 
print('size before:', testImages[10].shape)

# resize test images
test_images_resized = []
test_images_resized = [cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC) for img in testImages]

# check size after
print('size after:', test_images_resized[10].shape)
print()
print('number of test images:', len(test_images_resized) )

In [ ]:
# rescale 
if PREDICT_WITH_GEN == False:
    test_images_resized = np.array(test_images_resized)/255.

# reshape to 4D
test_images_resized = np.array(test_images_resized).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# check new shape
test_images_resized[0].shape

In [ ]:
del testImages
gc.collect()

### Inference on Test Images without generator

#test predictions without generator
test_preds = model.predict(test_images_resized, batch_size=BATCH_SIZE)

test_preds_class = np.argmax(test_preds, axis=1)
test_preds_class2 = [categories[test_preds_class[i]] for i in range(len(test_preds))] 

### Inference on Test images with generator

In [ ]:
#Augmentation config for Test Images

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_images_resized, batch_size=1)

test_preds_gen = model.predict_generator(test_generator, verbose=1, steps=len(test_images_resized))

In [ ]:
test_preds = np.argmax(test_preds_gen, axis=1)

In [ ]:
test_preds_gen[:3]

### Submission file

In [ ]:
mdl_name = 'ResNet50-1024-Radam'
out_name = 'Submission_'
out_name += mdl_name
out_name += '_epochs24'

sub_df = pd.read_csv(data_path/'submission_format.csv')
sub_df.head() 

In [ ]:
sub_df.iloc[:, 1:] = np.clip(test_preds_gen, a_min=0.05, a_max=0.95)

In [ ]:
# export to csv 
sub_df.to_csv('%s.csv' % out_name, index=None)

In [ ]:
sub_df.head()

### Fine-tune Transfer learning ResNet-50 

In [ ]:
# # fine tune ResNet-50 + Dense

# resnet.trainable = True
# set_trainable = False
# for layer in resnet.layers:
#     if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False
        
# # print df        
# layers_df = [(layer, layer.name, layer.trainable) for layer in resnet.layers]
# layers_df = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

# layers_df